In [75]:
from tda import auth, client
import json

token_path = 'C:/Users/Zachary/Desktop/token.pickle'
api_key = 'DRSMU4TL964FO3QNBQHVL78X9SUPGGIL@AMER.OAUTHAP'
redirect_uri = "http://localhost"
try:
    c = auth.client_from_token_file(token_path, api_key)
except FileNotFoundError:
    from selenium import webdriver
    with webdriver.Chrome() as driver:
        c = auth.client_from_login_flow(
            driver, api_key, redirect_uri, token_path)

In [ ]:
r = c.get_price_history('AAPL',
        period_type=client.Client.PriceHistory.PeriodType.YEAR,
        period=client.Client.PriceHistory.Period.TWENTY_YEARS,
        frequency_type=client.Client.PriceHistory.FrequencyType.DAILY,
        frequency=client.Client.PriceHistory.Frequency.DAILY)
assert r.status_code == 200, r.raise_for_status()
print(json.dumps(r.json(), indent=4))

In [85]:
p = client.Client.Account.Fields("positions")

In [86]:
a = c.get_accounts(fields=p)

In [87]:
assert a.status_code == 200, a.raise_for_status()
data = a.json()

In [88]:
import pandas as pd

In [89]:
dictd = data[0]

In [90]:
df = pd.json_normalize(dictd)

In [93]:
account_dataframes = []
for accountId, value in dictd.items():
    account_dataframes.append(pd.io.json.json_normalize(value))
    account_dataframes[-1].columns = [
        c.replace("securitiesAccount.", "")
        for c in account_dataframes[-1].columns
    ]

pd.concat(account_dataframes)

,type,accountId,roundTrips,isDayTrader,isClosingOnlyRestricted,positions,initialBalances.accruedInterest,initialBalances.availableFundsNonMarginableTrade,initialBalances.bondValue,initialBalances.buyingPower,...,currentBalances.bondValue,projectedBalances.availableFunds,projectedBalances.availableFundsNonMarginableTrade,projectedBalances.buyingPower,projectedBalances.dayTradingBuyingPower,projectedBalances.dayTradingBuyingPowerCall,projectedBalances.maintenanceCall,projectedBalances.regTCall,projectedBalances.isInCall,projectedBalances.stockBuyingPower
0,MARGIN,220535376,0,False,False,"[{'shortQuantity': 0.0, 'averagePrice': 42.43,...",0.0,3207.56,0.0,7214.82,...,0.0,3205.35,3205.35,7214.82,0.0,0.0,0.0,0.0,False,7214.82


In [101]:
pd.json_normalize(df["securitiesAccount.positions"][0])

,shortQuantity,averagePrice,currentDayProfitLoss,currentDayProfitLossPercentage,longQuantity,settledLongQuantity,settledShortQuantity,marketValue,maintenanceRequirement,instrument.assetType,instrument.cusip,instrument.symbol,instrument.description,instrument.putCall,instrument.underlyingSymbol
0,0.0,42.430,6.9500,2.85,5.0,5.0,0.0,250.7000,125.35,EQUITY,98422D105,XPEV,NaN,NaN,NaN
1,0.0,2.377,-16.5000,-6.98,1.0,1.0,0.0,220.0000,0.00,OPTION,0AAPL.OJ10127500,AAPL_031921P127.5,AAPL Mar 19 2021 127.5 Put,PUT,AAPL
2,1.0,3.113,19.0000,6.13,0.0,0.0,-1.0,-291.0000,250.00,OPTION,0AAPL.OJ10130000,AAPL_031921P130,AAPL Mar 19 2021 130.0 Put,PUT,AAPL
3,0.0,5.057,-14.5000,-6.29,1.0,1.0,0.0,216.0000,0.00,OPTION,0PTON.OJ10120000,PTON_031921P120,PTON Mar 19 2021 120.0 Put,PUT,PTON
4,1.0,6.443,25.0000,7.52,0.0,0.0,-1.0,-307.5000,500.00,OPTION,0PTON.OJ10125000,PTON_031921P125,PTON Mar 19 2021 125.0 Put,PUT,PTON
5,0.0,46.350,1.6155,0.51,5.0,5.0,0.0,315.8155,157.91,EQUITY,62914V106,NIO,NaN,NaN,NaN
6,1.0,2.443,-49.0000,-20.16,0.0,0.0,-1.0,-292.0000,0.00,OPTION,0AMD..CJ10100000,AMD_031921C100,AMD Mar 19 2021 100.0 Call,CALL,AMD
7,0.0,25.717,127.5000,5.54,1.0,1.0,0.0,2430.0000,0.00,OPTION,0AMD..AL20080000,AMD_012122C80,AMD Jan 21 2022 80.0 Call,CALL,AMD
8,1.0,0.293,-6.5000,-27.08,0.0,0.0,-1.0,-30.5000,190.50,OPTION,0RKT..OJ10016000,RKT_031921P16,RKT Mar 19 2021 16.0 Put,PUT,RKT
9,0.0,4.627,-37.5000,-8.93,1.0,1.0,0.0,382.5000,0.00,OPTION,0FB...OJ10250000,FB_031921P250,FB Mar 19 2021 250.0 Put,PUT,FB


In [105]:
direction = c.Movers.Direction("down")
change = c.Movers.Change("percent")

In [110]:
m = c.get_movers(index="$SPX.X", direction = direction, change = change)

In [111]:
m.json()

[{'change': -0.10095787064507925,
  'description': 'Akamai Technologies, Inc. - Common Stock',
  'direction': 'down',
  'last': 106.06,
  'symbol': 'AKAM',
  'totalVolume': 3389309},
 {'change': -0.08444275389882079,
  'description': 'Interpublic Group of Companies, Inc. (The) Common Stock',
  'direction': 'down',
  'last': 24.07,
  'symbol': 'IPG',
  'totalVolume': 2592882},
 {'change': -0.08117050532829145,
  'description': 'FMC Corporation Common Stock',
  'direction': 'down',
  'last': 106.915,
  'symbol': 'FMC',
  'totalVolume': 464473},
 {'change': -0.05717623974313238,
  'description': 'General Motors Company Common Stock',
  'direction': 'down',
  'last': 52.8547,
  'symbol': 'GM',
  'totalVolume': 20047029},
 {'change': -0.048107032703129056,
  'description': 'Tesla, Inc.  - Common Stock',
  'direction': 'down',
  'last': 808.595,
  'symbol': 'TSLA',
  'totalVolume': 8118560},
 {'change': -0.045830538788285205,
  'description': 'Incyte Corporation - Common Stock',
  'direction

In [1]:
import mibian as mb

In [65]:
c = mb.BS([468.3, 450, 1, 9], volatility=120)

In [60]:
c.putDelta

-0.37939599268979585

In [66]:
c.exerciceProbability

0.5472176928306169